In [4]:
%matplotlib inline
import matplotlib.pyplot as pl
import pystan
from pystan import StanModel

In [5]:
### Generate data from a Gaussian Process Simulator using PyStan:

# Simulator model:

# first model is a univariate model:

GP_simulator_model_1 = """

data {
  int<lower=1> N;
  real x[N];
}

transformed data {
  vector[N] mu;
  cov_matrix[N] Sigma;
  for (i in 1:N)
    mu[i] <- 0;
  for (i in 1:N)
    for (j in 1:N) {
      Sigma[i, j] <- exp(-pow(x[i] - x[j], 2)) + if_else(i==j, 0.1, 0.0);
      // The covariance matrix Sigma is not being computed efficiently here; 
      // see Section Section 15.3 of Stan Manual for a better approach.
    }      
}

parameters {
  vector[N] y;
}

model {
  y ~ multi_normal(mu, Sigma);
}

"""

# Compiled Stan Model
sm_sim_univariate = StanModel(model_code=GP_simulator_model_1)

In [8]:
# Second model is a multivariate model:

GP_simulator_model_2 = """

data {
  int<lower=1> D;
  int<lower=1> N;
  vector[D] x[N];
}

transformed data {
  vector[N] mu;
  cov_matrix[N] Sigma;
  for (i in 1:N)
    mu[i] <- 0;
  for (i in 1:N)
    for (j in 1:N) {
      Sigma[i, j] <- exp(-dot_self(x[i] - x[j])) + if_else(i==j, 0.1, 0.0);
      // The squared Euclidean distance calculation is done using the 
      // dot_self function, which returns the dot product of its argument 
      // with itself, here x[i] - x[j].
    }      
}

parameters {
  vector[N] y;
}

model {
  y ~ multi_normal(mu, Sigma);
}

"""

# Compiled Stan Model
sm_sim_multivariate = StanModel(model_code=GP_simulator_model_2)

In [11]:
# first model is a univariate model:

GP_simulator_model_3 = """

data {
  int<lower=1> N;
  real x[N];
}

transformed data {
  vector[N] mu;
  cov_matrix[N] Sigma;
  matrix[N, N] L;
  for (i in 1:N)
    mu[i] <- 0;
  for (i in 1:N)
    for (j in 1:N) {
      Sigma[i, j] <- exp(-pow(x[i] - x[j], 2)) + if_else(i==j, 0.1, 0.0);
      // The covariance matrix Sigma is not being computed efficiently here; 
      // see Section Section 15.3 of Stan Manual for a better approach.
    }  
    L <- cholesky_decompose(Sigma);
}

parameters {
  vector[N] z;
}

model {
  z ~ normal(0, 1);
}

generated quantities {
  vector[N] y;
  y <- mu + L * z;
}

"""

# Compiled Stan Model
sm_sim_univariate_with_Cholesky = StanModel(model_code=GP_simulator_model_3)

In [ ]:
import numpy as np

# Tutorial for DFM Goerge code. 
# Generate some fake noisy data.
N = 10
x = N * np.sort(np.random.rand(N))
yerr = 0.2 * np.ones_like(x)
y = np.sin(x) + yerr * np.random.randn(len(x))

print(x)
print(yerr)
print(y)

pl.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)



In [ ]:
GP_test_model = """

data {
  int<lower=1> N;
  vector[N] x;
  vector[N] y;
  }

transformed data {
  vector[N] mu;
  for (i in 1:N) 
    mu[i] <- 0;
  }

parameters {
  real<lower=0> eta_sq;
  real<lower=0> inv_rho_sq;
  real<lower=0> sigma_sq;
}

transformed parameters {
  real<lower=0> rho_sq;
  rho_sq <- inv(inv_rho_sq);
} 

model {
  matrix[N, N] Sigma;
  // off-diagonal elements
  for (i in 1:(N-1)) {
    for (j in (i+1):N) {
      Sigma[i, j] <- eta_sq * exp(-rho_sq * pow(x[i] - x[j],2));
      Sigma[j, i] <- Sigma[i, j];
    } 
  }
      // diagonal elements
  for (k in 1:N)
    Sigma[k, k] <- eta_sq + sigma_sq;  // + jitter

// hyperpriors: 
// Because the hyperparameters are required to be positive and expected to 
// have reasonably small values, broad half-Cauchy distribu- tions act as 
// quite vague priors which could just as well be uniform over a constrained 
// range of values.

//  eta_sq ~ cauchy(0, );
//  inv_rho_sq ~ cauchy(0, 5);
//  sigma_sq ~ cauchy(0, 5);

  eta_sq ~ gamma(1,1); // increasing alpha and beta makes variance smaller, but depends.
  inv_rho_sq ~ uniform(0, 2);
  sigma_sq ~ gamma(1, 1);


  y ~ multi_normal(mu, Sigma);
}
  
"""

# Compiled Stan Model
sm = StanModel(model_code=GP_test_model)

In [ ]:
data = {'N':N, 'x':x, 'y':y}

# Could set initial value to Max Likelihood solution without radius errors
#init = [{'lnf0':0.66,'alpha':-1.82,'beta':-0.65}]

fit = sm.sampling(data=data, iter=1000, chains=5, n_jobs=-1)

#get_inits(fit)

# Return a dictionary of arrays of posterior samples
la = fit.extract(permuted=True)  
#mu = la['mu']
#Sigma = la['Sigma']

eta_sq = la['eta_sq']
inv_rho_sq = la['inv_rho_sq']
sigma_sq = la['sigma_sq']
rho_sq = la['rho_sq']

a = fit.extract(permuted=False)
print(fit)



In [ ]:
import corner

corner.corner(np.hstack((eta_sq.reshape(-1,1), inv_rho_sq.reshape(-1,1), sigma_sq.reshape(-1,1), rho_sq.reshape(-1,1))), labels=[r"eta_sq", r"inv_rho_sq", r"sigma_sq", r"rho_sq"]);


In [ ]:
#print(sm)
print(eta_sq.shape)

In [ ]:
### calculate the mean for the random variable y 
#tilda for each vector of hyper parameters using the formula 
# for this.  Then plot the mean and std for each t value (value you want to predict). 

In [ ]:
print(fit)

In [ ]:
#print(la)

In [ ]:
#print(a)
#print(a.shape)

In [ ]:
fit.traceplot()

In [ ]:
##  Build a prediction function
import pylab as py
import pandas as pd

##### PREDICTION ####
 
# make a dataframe of parameter estimates for all chains
# params = pd.DataFrame({'eta_sq': fit.extract('eta_sq', permuted=True), 'sigma_sq': fit.extract('sigma_sq', permuted=True), , 'rho_sq': fit.extract('rho_sq', permuted=True)})


In [12]:
# Predictive Inference with a Gaussian Process:

GP_test_predict_model_1 = """

data {
  int<lower=1> N1;
  int<lower=1> N2;
  vector[N1] x1;
  vector[N1] y1;
  vector[N2] x2;
}

transformed data {
  int<lower=1> N;
  vector[N1+N2] x;
  vector[N1+N2] mu;
  cov_matrix[N1+N2] Sigma;
  N <- N1 + N2;
  for (n in 1:N1) 
    x[n] <- x1[n]; 
  for (n in 1:N2) 
    x[N1 + n] <- x2[n];
  for (i in 1:N) 
    mu[i] <- 0;
  for (i in 1:N)
    for (j in 1:N) {
      Sigma[i, j] <- exp(-pow(x[i] - x[j],2)) + if_else(i==j, 0.1, 0.0);
    }
}
    
parameters {
  vector[N2] y2;
}

model {
  vector[N] y;
  for (n in 1:N1) y[n] <- y1[n];
  for (n in 1:N2) y[N1 + n] <- y2[n];
  y ~ multi_normal(mu, Sigma);
}

"""

# Compiled Stan Model
sm_p = StanModel(model_code=GP_test_predict_model_1)

In [14]:
## Cholesky Factorization Speedup of above model:

GP_test_predict_model_2 = """

data {
  int<lower=1> N1;
  int<lower=1> N2;
  vector[N1] x1;
  vector[N1] y1;
  vector[N2] x2;
}

transformed data {
  int<lower=1> N;
  vector[N1+N2] x;
  vector[N1+N2] mu;
  cov_matrix[N1+N2] Sigma;
  matrix[N1+N2, N1+N2] L;
  N <- N1 + N2;
  for (n in 1:N1) 
    x[n] <- x1[n]; 
  for (n in 1:N2) 
    x[N1 + n] <- x2[n];
  for (i in 1:N) 
    mu[i] <- 0;
  for (i in 1:N)
    for (j in 1:N) {
      Sigma[i, j] <- exp(-pow(x[i] - x[j],2)) + if_else(i==j, 0.1, 0.0);
    }
  L <- cholesky_decompose(Sigma);
}
    
parameters {
  vector[N2] y2;
}

model {
  vector[N] y;
  for (n in 1:N1) y[n] <- y1[n];
  for (n in 1:N2) y[N1 + n] <- y2[n];
  y ~ multi_normal_cholesky(mu,L);  
}

"""

# Compiled Stan Model
sm_p_Cholesky_speedup = StanModel(model_code=GP_test_predict_model_2)

In [ ]:
# Generate some simple fake noisy data (from DFM Goerge tutorial).
N1 = 10
N2 = 500
x1 = N * np.sort(np.random.rand(N))
yerr = 0.2 * np.ones_like(x)
y1 = np.sin(x) + yerr * np.random.randn(len(x))
x2 = np.linspace(0, 10, 500)

#data_p = {'N':N, 'N1':N1, 'N2':N2, 'x1':x, 'x2':t, 'y1':y}
data_p = {'N1':N1, 'N2':N2, 'x1':x1, 'x2':x2, 'y1':y1}



In [ ]:
# syntax for initialization dictionary:
#init = [{'lnf0':0.66,'alpha':-1.82,'beta':-0.65}]

fit_p = sm_p.sampling(data=data_p, iter=10, chains=5, n_jobs=-1)

# Return a dictionary of arrays of posterior samples
la_p = fit_p.extract(permuted=True)  

y2 = la_p['y2']

a = fit_p.extract(permuted=False)

print(fit_p)